In [9]:
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', None)

In [10]:
df_stops = pd.read_csv('data/stops.txt')
df_routes = pd.read_csv('data/routes.txt')
df_stoptimes = pd.read_csv('data/stop_times.txt')
df_trips = pd.read_csv('data/trips.txt')
df_shapes = pd.read_csv('data/shapes.txt')
df_dates = pd.read_csv('data/calendar_dates.txt')

In [11]:
columns_to_drop = ['stop_headsign', 'stop_code', 'route_long_name', 'route_desc', 'route_color', 'route_text_color', 'trip_short_name']

In [12]:
result = pd.merge(df_stoptimes, df_stops, how='inner',on='stop_id')
result = pd.merge(result, df_trips, how='inner',on='trip_id')
result = pd.merge(result, df_routes, how='inner',on='route_id')
result = pd.merge(result, df_dates, how='inner',on='service_id')

In [13]:
result = result.drop(columns=columns_to_drop)

In [14]:
result['arrival_time_minutes'] = result['arrival_time'].map(lambda x: int(x[3:5]))
result['arrival_time_hours'] = result['arrival_time'].map(lambda x: int(x[:2]))
result['departure_time_minutes'] = result['departure_time'].map(lambda x: int(x[3:5]))
result['departure_time_hours'] = result['departure_time'].map(lambda x: int(x[:2]))

result['arrival_time_hours'] = result['arrival_time_hours'].apply(lambda x: x-24 if x >= 24 else x )
result['departure_time_hours'] = result['departure_time_hours'].apply(lambda x: x-24 if x >= 24 else x )

result['departure_time'] = result['departure_time'].apply(lambda x: str(int(x[:2]) - 24) + x[2:] if int(x[:2]) >= 24 else x)
result['arrival_time'] = result['arrival_time'].apply(lambda x: str(int(x[:2]) - 24) + x[2:] if int(x[:2]) >= 24 else x)

result.date = result.date.astype('str')
result['month'] = result['date'].map(lambda x: x[4:6])
result['day'] = result['date'].map(lambda x: x[6:8])
result['year'] = result['date'].map(lambda x: x[0:4])
result['date2'] = result['year'] + '-' +  result['month'] + '-' + result['day']

result['departure_time'] = result['date2'] + ' ' + result['departure_time']
result['arrival_time'] = result['date2'] + ' ' + result['arrival_time']

result['arrival_time'] = result['arrival_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
result['departure_time'] = result['departure_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

In [18]:
result

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_name,stop_lat,stop_lon,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,agency_id,route_short_name,route_type,date,exception_type,arrival_time_minutes,arrival_time_hours,departure_time_minutes,departure_time_hours,month,day,year,date2
0,401202301122334_152_401-01,2023-01-12 23:34:00,2023-01-12 23:34:00,195,0,0,1,Jaworzniaków 01,54.318041,18.568734,401,120230112,Osowa PKP,1,401_870029,1,0,N1,700,20230112,1,34,23,34,23,01,12,2023,2023-01-12
1,401202301122334_152_401-01,2023-01-12 23:35:00,2023-01-12 23:35:00,1959,1,3,3,Elfów 01,54.315644,18.569767,401,120230112,Osowa PKP,1,401_870029,1,0,N1,700,20230112,1,35,23,35,23,01,12,2023,2023-01-12
2,401202301122334_152_401-01,2023-01-12 23:36:00,2023-01-12 23:36:00,1957,2,3,3,Osiedle Świętokrzyskie 01,54.313707,18.571342,401,120230112,Osowa PKP,1,401_870029,1,0,N1,700,20230112,1,36,23,36,23,01,12,2023,2023-01-12
3,401202301122334_152_401-01,2023-01-12 23:38:00,2023-01-12 23:38:00,1790,3,3,3,Wieżycka 01,54.318050,18.587063,401,120230112,Osowa PKP,1,401_870029,1,0,N1,700,20230112,1,38,23,38,23,01,12,2023,2023-01-12
4,401202301122334_152_401-01,2023-01-12 23:39:00,2023-01-12 23:39:00,1320,4,3,3,Niepołomicka 01,54.320205,18.595057,401,120230112,Osowa PKP,1,401_870029,1,0,N1,700,20230112,1,39,23,39,23,01,12,2023,2023-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051286,4202301261601_32_004-14,2023-01-26 16:42:00,2023-01-26 16:42:00,2083,25,0,0,Żwirki i Wigury 01,54.392105,18.605795,4,220230126,Strzyża PKM,1,4_884487,1,0,4,900,20230126,1,42,16,42,16,01,26,2023,2023-01-26
2051287,4202301261601_32_004-14,2023-01-26 16:43:00,2023-01-26 16:43:00,2085,26,0,0,Jana Pawła II 01,54.394637,18.601580,4,220230126,Strzyża PKM,1,4_884487,1,0,4,900,20230126,1,43,16,43,16,01,26,2023,2023-01-26
2051288,4202301261601_32_004-14,2023-01-26 16:44:00,2023-01-26 16:44:00,2087,27,0,0,Startowa 01,54.397029,18.597542,4,220230126,Strzyża PKM,1,4_884487,1,0,4,900,20230126,1,44,16,44,16,01,26,2023,2023-01-26
2051289,4202301261601_32_004-14,2023-01-26 16:45:00,2023-01-26 16:45:00,2089,28,0,0,Zaspa 01,54.399350,18.594911,4,220230126,Strzyża PKM,1,4_884487,1,0,4,900,20230126,1,45,16,45,16,01,26,2023,2023-01-26


<h1>Functions</h1>

In [19]:
def get_all_departures_from_bus_stop(stop_id: int, arrival_time: str):
    time = datetime.strptime(arrival_time, '%H:%M')
    all_deparutres = result[(result["stop_id"]==stop_id) &
       (result['departure_time'] >= time )]
    return all_deparutres
#df.sort_values(by=['col1', 'col2'])

In [17]:
tmp = get_all_departures_from_bus_stop(313,'00:45')
tmp.sort_values(by=['departure_time_hours', 'departure_time_minutes'])

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_name,stop_lat,stop_lon,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,agency_id,route_short_name,route_type,date,exception_type,arrival_time_minutes,arrival_time_hours,departure_time_minutes,departure_time_hours,month,day,year,date2
67756,403202301120057_12_403-17,2023-01-12 00:57:00,2023-01-12 00:57:00,313,0,1,1,Wrzeszcz PKP 06,54.381766,18.603963,403,120230112,Dworzec Główny,1,403_903534,1,0,N3,700,20230112,1,57,0,57,0,01,12,2023,2023-01-12
154475,403202301130057_12_403-17,2023-01-13 00:57:00,2023-01-13 00:57:00,313,0,1,1,Wrzeszcz PKP 06,54.381766,18.603963,403,120230113,Dworzec Główny,1,403_903534,1,0,N3,700,20230113,1,57,0,57,0,01,13,2023,2023-01-13
219818,403202301140057_12_403-17,2023-01-14 00:57:00,2023-01-14 00:57:00,313,0,1,1,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Dworzec Główny,1,403_903534,1,0,N3,700,20230114,1,57,0,57,0,01,14,2023,2023-01-14
274520,403202301150057_12_403-17,2023-01-15 00:57:00,2023-01-15 00:57:00,313,0,1,1,Wrzeszcz PKP 06,54.381766,18.603963,403,120230115,Dworzec Główny,1,403_903534,1,0,N3,700,20230115,1,57,0,57,0,01,15,2023,2023-01-15
351550,403202301160057_12_403-17,2023-01-16 00:57:00,2023-01-16 00:57:00,313,0,1,1,Wrzeszcz PKP 06,54.381766,18.603963,403,120230116,Dworzec Główny,1,403_904324,1,0,N3,700,20230116,1,57,0,57,0,01,16,2023,2023-01-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813043,403202301222300_71_403-17,2023-01-22 23:53:00,2023-01-22 23:53:00,313,44,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230122,Wrzeszcz PKP,0,403_904329,1,0,N3,700,20230122,1,53,23,53,23,01,22,2023,2023-01-22
890073,403202301232300_71_403-17,2023-01-23 23:53:00,2023-01-23 23:53:00,313,44,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230123,Wrzeszcz PKP,0,403_904329,1,0,N3,700,20230123,1,53,23,53,23,01,23,2023,2023-01-23
975359,403202301242300_71_403-17,2023-01-24 23:53:00,2023-01-24 23:53:00,313,44,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230124,Wrzeszcz PKP,0,403_904329,1,0,N3,700,20230124,1,53,23,53,23,01,24,2023,2023-01-24
1060645,403202301252300_71_403-17,2023-01-25 23:53:00,2023-01-25 23:53:00,313,44,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230125,Wrzeszcz PKP,0,403_904329,1,0,N3,700,20230125,1,53,23,53,23,01,25,2023,2023-01-25
